In [1]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

In [4]:
def chooser_option_monte_carlo(S0, K, T1, T2, r, vol, N, M):
    """
    S0 : prix initial du sous-jacent
    K : prix d'exercice de l'option
    T1 : temps jusqu'à la date de choix (en mois)
    T2 : maturité totale de l'option (en années)
    r : taux sans risque
    vol : volatilité (supposée constante)
    N : nombre de pas de temps
    M : nombre de simulations
    """
    
    dt = T2 / N  # Intervalle de temps pour chaque timestep
    nudt = (r - 0.5 * vol**2) * dt
    volsdt = vol * np.sqrt(dt)
    
    # Nombre de pas pour atteindre la date de choix T1
    N1 = int(T1 / T2 * N)  # Conversion de T1 (en mois) en nombre de pas de temps
    
    sum_CT = 0
    sum_CT2 = 0

    for i in range(M):
        St = S0  # Réinitialisation du prix à chaque simulation
        CT = 0  # Réinitialiser CT à chaque simulation

        # Simulation du chemin du prix de l'actif jusqu'à la maturité
        for j in range(N):
            epsilon = np.random.normal()  # Variable aléatoire pour simuler le mouvement brownien
            St *= np.exp(nudt + volsdt * epsilon)
            
            # À la date de choix, calcul du payoff
            if j == N1:
                # Calcul de Sf (prix forward à la date de choix)
                Sf = S0 * np.exp(r * (T2 - T1/12))  # Calcul correct de Sf
                # Calcul des payoffs avec Sf au lieu de K
                payoff_call = max(0, St - Sf)  # Payoff du call
                payoff_put = max(0, Sf - St)   # Payoff du put
                CT = max(payoff_call, payoff_put)  # Choisir l'option la plus favorable (call ou put)

        # À la maturité, ajout du payoff du call basé sur K
        payoff_final = max(0, St - K)  # Payoff final du call à l'échéance
        CT = max(CT, payoff_final)  # Choisir le meilleur payoff entre la date de choix et l'échéance

        # Accumuler les résultats des simulations
        sum_CT += CT
        sum_CT2 += CT**2

    # Calcul de la valeur présente de l'option
    C0 = np.exp(-r * T2) * sum_CT / M  # Moyenne des payoffs actualisés
    # Calcul de l'écart-type des payoffs pour l'estimation de l'erreur standard
    sigma = np.sqrt((sum_CT2 - (sum_CT**2) / M) * np.exp(-2 * r * T2) / (M - 1))
    SE = sigma / np.sqrt(M)  # Erreur standard

    return C0

In [5]:
chooser_option_monte_carlo(100, 100, 6, 2, 0.04, 0.2, 100, 1000)

14.329205247999132